In [4]:
import requests
import pandas as pd

year = 2020
level_of_study = 99

base_url = f"https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/{year}/{level_of_study}/race/sex/"

all_rows = []
url = base_url

while url:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    j = r.json()
    
    all_rows.extend(j["results"])
    url = j.get("next")   # pagination

df = pd.DataFrame(all_rows)

print("Total rows:", len(df))
print(df.head())



Total rows: 500730
   unitid  year  ftpt  fips  level_of_study  degree_seeking  class_level  sex  \
0  100654  2020     1     1              99              99           99    1   
1  100663  2020     1     1              99              99           99    1   
2  100690  2020     1     1              99              99           99    1   
3  100706  2020     1     1              99              99           99    1   
4  100724  2020     1     1              99              99           99    1   

   race  enrollment_fall  
0     1               43  
1     1             3154  
2     1               66  
3     1             3113  
4     1               56  


In [5]:
df.to_csv(r'C:\Users\visha\OneDrive\Desktop\Data Projects\University student analytics mini data warehouse\Data\Raw\ipeds_fall_enrollment_2020_raw.csv', index= False)

In [6]:


year = 2020

# The API uses .../api/v1/{topic}/{source}/{endpoint}/{year}/...  :contentReference[oaicite:1]{index=1}
# Different portals sometimes label retention endpoint slightly differently.
candidates = [
    f"https://educationdata.urban.org/api/v1/college-university/ipeds/fall-retention/{year}/",
    f"https://educationdata.urban.org/api/v1/college-university/ipeds/retention/{year}/",
    f"https://educationdata.urban.org/api/v1/college-university/ipeds/fall-retention-rate/{year}/",
]

working_url = None
last_error = None

for url in candidates:
    try:
        r = requests.get(url, timeout=60)
        if r.status_code == 200:
            working_url = url
            break
        else:
            last_error = f"{r.status_code} for {url}"
    except Exception as e:
        last_error = str(e)

if not working_url:
    raise RuntimeError(f"Could not find a working retention endpoint. Last error: {last_error}")

print(" Using endpoint:", working_url)

# Pagination pull (same pattern you used for enrollment)
all_rows = []
url = working_url

while url:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    j = r.json()
    all_rows.extend(j["results"])
    url = j.get("next")

df_ret = pd.DataFrame(all_rows)

print("Total rows:", len(df_ret))
print("Columns:", df_ret.columns.tolist())
print(df_ret.head())

 Using endpoint: https://educationdata.urban.org/api/v1/college-university/ipeds/fall-retention/2020/
Total rows: 17508
Columns: ['unitid', 'year', 'fips', 'ftpt', 'retention_rate', 'returning_students', 'prev_cohort', 'prev_exclusions', 'prev_cohort_adj']
   unitid  year  fips  ftpt  retention_rate  returning_students  prev_cohort  \
0  100654  2020     1     1            0.54               911.0       1688.0   
1  100654  2020     1     2            0.33                 2.0          6.0   
2  100654  2020     1    99            0.54               913.0       1694.0   
3  100663  2020     1     1            0.86              1982.0       2294.0   
4  100663  2020     1     2            0.48                20.0         42.0   

   prev_exclusions  prev_cohort_adj  
0              2.0           1686.0  
1              0.0              6.0  
2              2.0           1692.0  
3              0.0           2294.0  
4              0.0             42.0  


In [7]:
df_ret.to_csv(
    r"C:\Users\visha\OneDrive\Desktop\Data Projects\University student analytics mini data warehouse\Data\Raw\ipeds_retention_2020_raw.csv",
    index=False
)
